In [1]:
import numpy as np
import os
import tensorflow
import pandas as pd

In [2]:
# Path for exported data, numpy arrays
DATA_PATH = r"C:\Users\maria\Documents\MP_Data"

# Actions that we try to detect
actions = np.array(
    ['a', 'ą', 'b', 'c', 'ć', 'ch', 'cz', 'd', 'e', 'ę', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'ł', 'm', 'n', 'ń', 'o',
     'ó', 'p', 'r', 'rz', 's', 'ś', 'sz', 't', 'u', 'w', 'y', 'z', 'ź', 'ż', 'q', 'v', 'x'])

# Two hundred videos worth of data
no_sequences = 200

# Videos are going to be 30 frames in length
sequence_length = 20

In [3]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

label_map = {label: num for num, label in enumerate(actions)}

sequences2, labels2 = [], []
for action in actions:
    print(action)
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences2.append(window)
        labels2.append(label_map[action])
    print(action+' completed')

a
a completed
ą
ą completed
b
b completed
c
c completed
ć
ć completed
ch
ch completed
cz
cz completed
d
d completed
e
e completed
ę
ę completed
f
f completed
g
g completed
h
h completed
i
i completed
j
j completed
k
k completed
l
l completed
ł
ł completed
m
m completed
n
n completed
ń
ń completed
o
o completed
ó
ó completed
p
p completed
r
r completed
rz
rz completed
s
s completed
ś
ś completed
sz
sz completed
t
t completed
u
u completed
w
w completed
y
y completed
z
z completed
ź
ź completed
ż
ż completed
q
q completed
v
v completed
x
x completed


In [23]:
x = np.array(sequences2)
y = to_categorical(labels2).astype(int)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=2019)

print('TRAIN:',x_train.shape, y_train.shape)
print('TEST:',x_test.shape, y_test.shape)
print('VALIDATION:',x_val.shape, y_val.shape)

TRAIN: (5460, 20, 366) (5460, 39)
TEST: (1170, 20, 366) (1170, 39)
VALIDATION: (1170, 20, 366) (1170, 39)


In [5]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard

In [6]:
from datetime import datetime

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tb_callback = tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [24]:
# Hiperparametry do przeszukiwania
param_grid = {
    'batch_size': [16, 32, 64],
    'epochs': [50, 70, 100],
    'learning_rate': [0.001, 0.01, 0.05]
}

results2 = []

for epoch_number in (param_grid['epochs']):
    for learning_rate_number in (param_grid['learning_rate']):
        for batch_size_number in (param_grid['batch_size']):
            model = Sequential()
            model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(20, 366)))
            model.add(LSTM(128, return_sequences=True, activation='relu'))
            model.add(LSTM(64, return_sequences=False, activation='relu'))
            model.add(Dense(64, activation='relu'))
            model.add(Dense(32, activation='relu'))
            model.add(Dense(actions.shape[0], activation='softmax'))
            optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_number)
            model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
            #model.fit(x_train, y_train, epochs=epoch_number, callbacks=[tb_callback])
            #model.fit(x_train, y_train, epochs=epoch_number, batch_size=batch_size_number, verbose=0)

            # Training the model with validation data
            history = model.fit(x_train, y_train,
                                epochs=epoch_number,
                                batch_size=batch_size_number,
                                validation_data=(x_val, y_val),
                                callbacks=[tf.keras.callbacks.TensorBoard(log_dir=f"logs/fit/{datetime.now().strftime('%Y%m%d-%H%M%S')}", histogram_freq=1)])

             # Predict on train and test sets
            y_train_pred = model.predict(x_train)
            y_test_pred = model.predict(x_test)

            # Convert predictions from probabilities to class labels
            y_train_pred_class = np.argmax(y_train_pred, axis=1)
            y_test_pred_class = np.argmax(y_test_pred, axis=1)
            y_train_class = np.argmax(y_train, axis=1)
            y_test_class = np.argmax(y_test, axis=1)

            # Calculate metrics
            accuracy_train = accuracy_score(y_train_class, y_train_pred_class)
            accuracy_test = accuracy_score(y_test_class, y_test_pred_class)
            recall_train = recall_score(y_train_class, y_train_pred_class, average='macro')
            recall_test = recall_score(y_test_class, y_test_pred_class, average='macro')
            f1_train = f1_score(y_train_class, y_train_pred_class, average='macro')
            f1_test = f1_score(y_test_class, y_test_pred_class, average='macro')

            # Append scores to DataFrame
            results2.append({
                "epochs": epoch_number,
                "learning_rate": learning_rate_number,
                "batch_size": batch_size_number,
                "accuracy_train": accuracy_train,
                "accuracy_test": accuracy_test,
                "recall_train": recall_train,
                "recall_test": recall_test,
                "f1_train": f1_train,
                "f1_test": f1_test
            })

            print ('Batch_size: ' + str(batch_size_number) + '\n')
            print ('Epochs: ' + str(epoch_number) + '\n')
            print ('Learning_rate: ' + str(learning_rate_number) + '\n')
            print ('Accuracy_test_rate: ' + str(accuracy_test) + '\n')

results2 = pd.DataFrame(results2)

# Save scores to CSV
results2.to_csv('model_scores2.csv', index=False)


C:\Users\maria\PycharmProjects\AUM_projekt_II_stp\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
342/342 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - categorical_accuracy: 0.0328 - loss: 3.6704 - val_categorical_accuracy: 0.0590 - val_loss: 3.4911
Epoch 2/50
342/342 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - categorical_accuracy: 0.0572 - loss: 3.3939 - val_categorical_accuracy: 0.0675 - val_loss: 3.4191
Epoch 3/50
342/342 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - categorical_accuracy: 0.1146 - loss: 3.0880 - val_categorical_accuracy: 0.1897 - val_loss: 2.6849
Epoch 4/50
342/342 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - categorical_accuracy: 0.1870 - loss: 2.6338 - val_categorical_accuracy: 0.2940 - val_loss: 2.3183
Epoch 5/50
342/342 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - categorical_accuracy: 0.2372 - loss: 2.4141 - val_categorical_accuracy: 0.3120 - val_loss: 2.1812
Epoch 6/50
342/342 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - categorical_accuracy: 0.3119 - loss: 2.2046 - val_categorical_accuracy: 0.3274 - val_loss: 2.2220
Epoch 7/50
342/342 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - categorical_accuracy: 0.38

C:\Users\maria\PycharmProjects\AUM_projekt_II_stp\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - categorical_accuracy: 0.0276 - loss: 3.6570 - val_categorical_accuracy: 0.0368 - val_loss: 3.6677
Epoch 2/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - categorical_accuracy: 0.0283 - loss: 3.6741 - val_categorical_accuracy: 0.0222 - val_loss: 3.6600
Epoch 3/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - categorical_accuracy: 0.0304 - loss: 3.6796 - val_categorical_accuracy: 0.0282 - val_loss: 3.6654
Epoch 4/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - categorical_accuracy: 0.0310 - loss: 3.7327 - val_categorical_accuracy: 0.0205 - val_loss: 3.6676
Epoch 5/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - categorical_accuracy: 0.0305 - loss: 3.6632 - val_categorical_accuracy: 0.0205 - val_loss: 3.6679
Epoch 6/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - categorical_accuracy: 0.0258 - loss: 3.6626 - val_categorical_accuracy: 0.0205 - val_loss: 3.6681
Epoch 7/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - categorical_accuracy: 0.030

C:\Users\maria\PycharmProjects\AUM_projekt_II_stp\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

342/342 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - categorical_accuracy: 0.0234 - loss: 31019004.0000 - val_categorical_accuracy: 0.0350 - val_loss: 3.6810
Epoch 2/100
342/342 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - categorical_accuracy: 0.0274 - loss: 3.6802 - val_categorical_accuracy: 0.0291 - val_loss: 3.6834
Epoch 3/100
342/342 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - categorical_accuracy: 0.0274 - loss: 3.6821 - val_categorical_accuracy: 0.0214 - val_loss: 3.6839
Epoch 4/100
342/342 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - categorical_accuracy: 0.0228 - loss: 3.6870 - val_categorical_accuracy: 0.0222 - val_loss: 3.6846
Epoch 5/100
342/342 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - categorical_accuracy: 0.0274 - loss: 3.6821 - val_categorical_accuracy: 0.0299 - val_loss: 3.6897
Epoch 6/100
342/342 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - categorical_accuracy: 0.0268 - loss: 3.6792 - val_categorical_accuracy: 0.0282 - val_loss: 3.6842
Epoch 7/100
342/342 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - categorical_accuracy: 0.

C:\Users\maria\PycharmProjects\AUM_projekt_II_stp\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


171/171 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - categorical_accuracy: 0.0204 - loss: 9909247.0000 - val_categorical_accuracy: 0.0222 - val_loss: 3.6769
Epoch 2/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - categorical_accuracy: 0.0264 - loss: 3.6757 - val_categorical_accuracy: 0.0291 - val_loss: 3.6852
Epoch 3/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - categorical_accuracy: 0.0304 - loss: 3.6710 - val_categorical_accuracy: 0.0248 - val_loss: 3.6732
Epoch 4/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - categorical_accuracy: 0.0276 - loss: 3.6739 - val_categorical_accuracy: 0.0214 - val_loss: 3.6696
Epoch 5/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - categorical_accuracy: 0.0254 - loss: 3.6778 - val_categorical_accuracy: 0.0274 - val_loss: 3.6764
Epoch 6/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - categorical_accuracy: 0.0283 - loss: 3.6767 - val_categorical_accuracy: 0.0299 - val_loss: 3.6774
Epoch 7/100
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - categorical_accuracy: 0.02

C:\Users\maria\PycharmProjects\AUM_projekt_II_stp\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


86/86 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - categorical_accuracy: 0.0288 - loss: 17015362.0000 - val_categorical_accuracy: 0.0274 - val_loss: 3.6721
Epoch 2/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - categorical_accuracy: 0.0270 - loss: 3.6704 - val_categorical_accuracy: 0.0231 - val_loss: 3.6711
Epoch 3/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - categorical_accuracy: 0.0265 - loss: 3.6699 - val_categorical_accuracy: 0.0256 - val_loss: 3.6712
Epoch 4/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - categorical_accuracy: 0.0237 - loss: 3.6699 - val_categorical_accuracy: 0.0205 - val_loss: 3.6728
Epoch 5/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - categorical_accuracy: 0.0264 - loss: 3.6685 - val_categorical_accuracy: 0.0325 - val_loss: 3.6765
Epoch 6/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - categorical_accuracy: 0.0265 - loss: 3.6711 - val_categorical_accuracy: 0.0214 - val_loss: 3.6735
Epoch 7/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - categorical_accuracy: 0.0222 - loss: 3.

In [22]:
print(results)

    epochs  learning_rate  batch_size  accuracy_train  accuracy_test  \
0       10          0.001          16        0.027839       0.020513   
1       10          0.001          32        0.400916       0.395299   
2       10          0.010          16        0.028022       0.020085   
3       10          0.010          32        0.027839       0.020513   
4       50          0.001          16        0.774542       0.718376   
5       50          0.001          32        0.841209       0.784615   
6       50          0.010          16        0.026007       0.024786   
7       50          0.010          32        0.027839       0.020513   
8      100          0.001          16        0.202930       0.201282   
9      100          0.001          32        0.028205       0.019658   
10     100          0.010          16        0.028022       0.020085   
11     100          0.010          32        0.028205       0.019658   

    recall_train  recall_test  f1_train   f1_test  
0       0.0